# King County Residential Property Values

Gavin Peterkin

## Introduction

Regardless of one's background or living situation, the housing market affects us all, but like most social and economic systems, it seems too complex to make any sense of. My aim in this project is to make use of the increasingly large public data sets made accessible by Seattle and King County to investigate the factors that influence a residential property's value over time. In effect, I seek to model and understand how features from 'number of bedrooms' to proximity to private schools and even to whether or not a property has a view of the Seattle skyline has any influence on that property's value and if so, by how much. Since tastes change over time, and I have access to sales data over a large period of time, I will also investigate how people's preferences are changing and extend that trend to the future to see what it _will_ mean for Seattle's booming housing market over the next couple of years.

* Note: I was inspired partly by the [Kaggle home prices](https://www.kaggle.com/c/house-prices-advanced-regression-techniques) competition.

## Data Preparation Workflow

First, I downloaded raw csv files from the sources listed below.

### Data Sources
1. [King County Assessor Data](http://info.kingcounty.gov/Assessor/DataDownload/default.aspx): Has many features from tax assessor data including: recent sales prices, assessed values, slope of terrain, etc.
2. [King County GIS Center](http://www.kingcounty.gov/services/gis.aspx): Has parcel geographic information systems (GIS) data including: parcel location and shape for multiple property types
3. [Seattle Open Data](https://data.seattle.gov/): Has Latitudes and Longitudes for things like schools, libraries, parks, etc.

After downloading the files, I developed scripts in the Bourne Again Shell (BASH) and/or python to automate (or partially automate) the process of cleaning the files and putting them into a PostgreSQL (PSQL) database on a remote server. There are significant speed limitations as a result of using a remote database, but it also allows for easy extensibility in the future. As more data become available, it can simply be dumped in, and the model can be re-run.

Another benefit of PSQL is that there's an actively developed extension [PostGIS](http://www.postgis.net/) that allows for the creation of spatially/geographically enabled databases. This allows for relatively fast spatial computations that would be prohibitively slow if done in python.

For more detail about how I pushed the data to the database and the issues I encountered along the way, see the notebook `csvs_to_psql` and the bash scripts `file_prep.bash` and `shp_processor.bash`. Shape files are an unusual file type. Luckily Postgis has functionality to ease the transition to PSQL. Still, I encountered a myriad of problems along the way mostly dealing with incorrect data types. I tried to fix these as they arose, but it was challenging.


### Feature Engineering

Finally, I had to do a lot of filtering, joining, and spatial operations in sql to get the final residential table. (In the sql directory, see `queries.sql`.) I generated a lot of the sql in the query file in the notebook `sql_generator`. This is where I create features like 'number of private schools in an x-meter radius'. This has been an iterative process. I'm continually updating the complete residential table as I discover issues or more useful features to add.

As part of the feature engineering process, I have been reducing the dimensionality of several sets of features like the 'View' features. I used principal comonent analysis (PCA) for this process, but it's not clear if the results are actually better than keeping the original features intact. For one thing, it makes the results less interpretable.

For categorical variables, namely the district, I created dummy variables. I think a better route in the future would be to create my own "districts" from latitude and longitude and try to cluster properties based on their assessed values.

## Exploratory Data Analysis Findings

Please see the [exploratory analysis notebook](./exploratory_analysis.ipynb) for an overview of some of the initial findings.

Overall, there are about 50,000 properties with sale prices that range from zero to several million dollars.

After filtering to recent sales that were determined to be 'good' (within a certain range from the assessed value and sold recently), there were around 4,000 properties to model.

## Feature Selection

There are about 65 features in the original data. Based on the analysis in the [feature development notebook](./feature_development.ipynb), I hand selected seem to be correlated in some way with price

## Model Selection



## Conclusion

* Don't reject the hypothesis that the housing features I selected influence sale price.

## Next steps and areas for improvement

### New features
There are a lot of characteristics that this model simply isn't capturing. New features like a neighborhood "livability" score, affordability characteristics, employment center proximity, traffic, and store proximity are all very reasonable features to include in the next iteration of the model.

### Feature selection
Using L1 and L2 regularization techniques via Lasso and Ridge regressions and selecting features by hand seems a little primitive. It also does a poor job of distinguishing features that are highly correlated. Using random forests (RF) to rank features in importance could be another useful step in determining which features are useful especially as more and more features are added. I can also reduce the dimensionality of highly correlated features.

### Model improvement
The biggest area for improvement is in the modeling process.